#### 1. spring framework 有几大模块 
* `spring core`
* `spring Bean`:    
  核心组件 BeanFactory, 是工厂模式的实现. 使用控制反转 (IOC) 把应用程序的配置和对象依赖跟实际的应用代码分开  
* `spring context`:   
  主要包含各种 ApllicationContext, 校验, 调度等
* `spring aop`:   
  面向切面编程, 让 spring 容器管理的任何对象支持 AOP
* `spring tx`:    
  声明式事务
* `spring test`

###  2. Spring 用到的设计模式  (??)



#### 3. Spring 提供几种注入方式?   (详读DI部分)
Spring 支持2中注入方式: `构造器注入`和`setter注入`

    |构造函数注入|setter 注入|
    |--------|-------|
    |没有部分注入|有部分注入|
    |不会覆盖 setter 属性|会覆盖 setter 属性|
    |任意修改都会创建一个新实例|	任意修改不会创建一个新实例|
    |适用于设置很多属性|	适用于设置少量属性|

实际场景下, setter 注入用的更多  


#### 4. Spring 有几种IoC 容器?   (??)
有2中: `BeanFactory` 和 `ApplicationContext`
1. spring 文档中 关于 beanfactory 和 applicationCOntext 的区别  
2. [http://svip.iocoder.cn/Spring/ApplicationContext/] (applicationCOntext 源码)


#### 5. 有哪些常用的 ApplicationContext 容器？
1. `ClassPathXmlApplicationContext`   
   从 ClassPath 的 XML 配置文件中读取上下文，并生成上下文定义。应用程序上下文从程序环境变量中取得。示例代码如下：
    ```java
    ApplicationContext context = new ClassPathXmlApplicationContext(“bean.xml”);
    ```
2. `ConfigServletWebServerApplicationContext`   
   因为现在用 spring-boot 多, 所以基本都是用这个容器.

### 6. 简述 spring IoC 的实现    (???)
简单来说就是反射 + 工厂模式 
[https://www.iocoder.cn/Fight/Interview-poorly-asked-Spring-IOC-process-1/](https://www.iocoder.cn/Fight/Interview-poorly-asked-Spring-IOC-process-1/)


#### 7. spring framework 中有哪些事件   
如果一个 bean 实现了 `ApplicationListener` 接口, 则当一个 `ApplicationEvent` (抽象类)被发布以后，Bean 会自动被通知.    
spring 提供了5种基本事件: 
* Context 开始事件: 调用 ConfigurableApplicationContext 的 start() 方法开启容器时   
* Context 停止事件: 调用 ConfigurableApplicationContext 的 stop() 方法时   
* Context 关闭事件: 容器被关闭时, 所有单例 bean 被销毁
* Context 更新事件: 调用 ConfigurableApplicationContext 的 refresh() 方法时   
* 请求处理事件(RequestHandledEvent): web 应用中, 当一个 request 请求结束时触发该事件  
* spring 还可以自定义事件  
    ```java
    // 1. 继承 ApplicationEvent 抽象类
    public class CustomApplicationEvent extends ApplicationEvent{  
        public CustomApplicationEvent(Object source, final String msg) {  
            super(source);
        }  
    }
    // 2. 让 bean 实现 ApplicationListener 接口
    public class CustomEventListener implements ApplicationListener<CustomApplicationEvent> {
        @Override  
        public void onApplicationEvent(CustomApplicationEvent applicationEvent) {  
            // handle event  
        }
    }
    // 3. 让容器发布事件, 从而让 bean 监听到事件  
    applicationContext.publishEvent(customEvent);
    ```


### 8. Spring Bean 在容器的生命周期是什么样的   (??)
**这道题最好结合源码说 (精进源码系列)** 
[http://svip.iocoder.cn/categories/Spring/](http://svip.iocoder.cn/categories/Spring/)
1. spring bean 的初始化流程
    1. 实例化 bean 对象
        1. 根据配置的 `BeanDefinition` 实例化对象  (`BeanDefinition`可以来自 xml 文件, java 注解, java config)
        2. 完成 bean 属性的依赖注入. 先进行构造器注入, 在进行 setter 注入 
    2. 根据情况, 注入 aware 相关属性   
        1. 如果 bean 实现了 `BeanNameAware` 接口, 则工厂通过接口的 `setBeanName()` 方法注入 beanName
        2. 如果 bean 实现了 `BeanFactoryAware` 接口, 则工厂通过传递自身实例, 来调用 `setBeanFactory(BeanFactory beanFactory)` 方法
    3. 调用 `BeanPostProcessor` 接口的 `preProcessBeforeInitialization(Object bean, String beanName)` 方法
    4. 调用 `InitializingBean` 接口的 `afterPropertiesSet()` 方法
    5. 如果为 bean 指定了 init 方法（例如 `<bean/> 的 init-method 属性`），那么将调用该方法。
    5. 调用 `BeanPostProcessor` 接口的 `postProcessBeforeInitialization(Object bean, String beanName)` 方法
    
2. spring bean 的销毁流程
    1. 如果 bean 实现 `DisposableBean` 接口，当 spring 容器关闭时，会调用 `destroy()` 方法。
    2. 如果 bean 指定了 destroy 方法（例如 `<bean/> 的 destroy-method 属性`），那么将调用该方法。
    
下面用一张图片概括 bean 的初始化 和 销毁流程   
<img src="img/beanLifeCycle.png">

#### 9. spring 有哪些装配方式 (??)  
每种注解对应什么装配方式
* byName 装配:  
* byType 装配: 


#### 10. 什么是延迟加载   
spring 容器在启动时会以单例模式创建所有配置的 bean, 如果想要在使用 bean 时才创建, 就要声明 `lzay-init = "true"`.    
启动时创建 bean  可以让 spring 检查是否存在配置错误, 做到第一时间发现错误

#### 11. spring 的 单例 bean 是线程安全的吗   
spring 不对 bean 的线程安全性做处理, 所以不能算线程安全的. 但是业务中, 设置为 singleton 的 bean 一般都是无状态的, 所以也就不存在线程安全问题. 如果 bean 是有状态的, 将作用于改为 prototype 即可  

### 12. spring bean 如何解决循环依赖 (??)  
结合源码 [http://svip.iocoder.cn/Spring/IoC-get-Bean-createBean-5/](http://svip.iocoder.cn/Spring/IoC-get-Bean-createBean-5/)

###  进入 spring 注解部分

#### 13. @Component, @Controller, @Repository, @Service 有何区别？
* @Component ：它将 Java 类标记为 Bean 。它是任何 Spring 管理组件的通用构造型。
* @Controller ：它将一个类标记为 Spring Web MVC 控制器。
* @Service ：此注解是组件注解的特化。它不会对 @Component 注解提供任何其他行为。您可以在服务层类中使用 @Service 而不是 @Component ，因为它以更好的方式指定了意图。
* @Repository ：这个注解是具有类似用途和功能的 @Component 注解的特化。它为 DAO 提供了额外的好处。它将 DAO 导入 IoC 容器，并使未经检查的异常有资格转换为 Spring DataAccessException 。

#### 14. @Autowired 注解与其它注解的配合有什么用？ (??)  
spring doc 中的记载
@Autowired 注解，可以更准确地控制应该在何处以及如何进行自动装配。

1. 默认的 **byType** 注解  
   此注解用于在 setter 方法，构造函数，具有任意名称或多个参数的属性或方法上自动装配 Bean。  
   默认情况下，它是类型驱动的注入。
    ```java
    public class EmpAccount {

        @Autowired
        private Employee emp;

    }
    ```
2. 配合 `@Qualifier` 实现 **byId** 注解  
   例如，应用中有两个类型为 Employee 的 Bean ID 为 "emp1" 和 "emp2" ，此处，我们希望 EmployeeAccount Bean 注入 "emp1" 对应的 Bean 对象。代码如下：
    ```java
    public class EmployeeAccount {

        @Autowired
        @Qualifier(emp1)
        private Employee emp;

    }
    ```
3. ... 

### Spring AOP 相关  
[理论](https://segmentfault.com/a/1190000007469968)
[实战](https://segmentfault.com/a/1190000007469982)

### Spring 事务

spring 事务有两种类型 

日常使用:   
[spring boot + 注解的声明式事务处理](https://www.jianshu.com/p/cddeca2c9245)
[](https://blog.csdn.net/xktxoo/article/details/77919508)